In [1]:
import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

In [2]:
costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

In [3]:
pp.runopp(net, delta=1e-16)

In [4]:
net.res_ext_grid

,p_mw,q_mvar
0,56.530584,1.974564


In [5]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,71.309255,-1.969681,-3.713031,1.000008
1,12.303443,-1.451180,-3.712735,1.000010


In [6]:
net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 12

In [7]:
pp.runopp(net, delta=1e-16)

In [8]:
net.res_ext_grid

,p_mw,q_mvar
0,144.559166,9.193021


In [9]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000079,8.601766,-16.426835,0.967619
1,0.000225,10.594623,-13.481007,0.989756


In [10]:
net.res_cost

1445.595544859482

In [11]:
net.res_trafo.loading_percent

0    144.851179
Name: loading_percent, dtype: float64

In [12]:
net.trafo["max_loading_percent"] = 50

In [13]:
pp.runopp(net, delta=1e-16)

In [14]:
net.res_trafo.loading_percent

0    49.999136
Name: loading_percent, dtype: float64

In [15]:
net.res_ext_grid

,p_mw,q_mvar
0,49.953012,-2.147126


In [16]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000598,2.992989,-6.232710,0.985230
1,93.304317,3.453173,-1.237784,1.025709


In [17]:
net.res_cost

1619.1908981410584

In [18]:
net.res_line.loading_percent

0    22.192863
1    57.476322
2    33.473473
Name: loading_percent, dtype: float64

In [19]:
net.line["max_loading_percent"] = 50
pp.runopp(net, delta=1e-16)

In [20]:
net.line["max_loading_percent"] = 50
pp.runopp(net, delta=1e-16)

In [21]:
net.res_line.loading_percent

0    18.905589
1    49.999986
2    30.435895
Name: loading_percent, dtype: float64

In [22]:
net.res_ext_grid

,p_mw,q_mvar
0,49.787584,-4.603451


In [23]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,9.136446,2.430957,-5.815440,0.993015
1,83.592614,4.853496,-1.511326,1.028887


In [24]:
net.res_cost

1638.0339026783786

In [25]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-49.787584,4.603451,10.000000,-1.410210e-21
1,1.006024,-3.408832,60.000000,0.000000,13.095255,-5.409162e-02
2,0.993015,-5.815440,60.863554,-2.430957,14.999985,4.490949e-22
3,1.028887,-1.511326,-73.592614,-4.853496,12.000007,1.781412e-21


In [26]:
net.bus["min_vm_pu"] = 1.0
net.bus["max_vm_pu"] = 1.02
pp.runopp(net, delta=1e-16)

In [27]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-49.906839,3.050610,10.000000,-2.360897e-22
1,1.004168,-3.421014,60.000000,0.000000,13.126862,-2.133418e-02
2,1.000000,-5.976093,59.278168,-14.859011,14.999997,1.148963e-21
3,1.020000,-1.366891,-71.863462,9.172781,12.000002,-7.089262e-22


In [28]:
net.res_ext_grid

,p_mw,q_mvar
0,49.906839,-3.05061


In [29]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,10.721832,14.859011,-5.976093,1.00
1,81.863462,-9.172781,-1.366891,1.02


DC-OPF

In [30]:
pp.rundcopp(net, delta=1e-16)

In [31]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-50.000000,3.050610,10.000000,0.0
1,1.004168,-3.436967,60.000000,0.000000,13.090909,0.0
2,1.000000,-5.708566,61.488746,-14.859011,15.000000,0.0
3,1.020000,-1.362340,-71.488747,9.172781,12.000000,0.0


In [32]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,16.715233,0.0,-16.715233,0.0,0.0,0.0,0.087732,0.087732,0.087732,1.0,-3.436967,1.0,-5.708566,18.666430
1,-44.773513,0.0,44.773513,0.0,0.0,0.0,0.235000,0.235000,0.235000,1.0,-5.708566,1.0,-1.362340,50.000000
2,26.715233,0.0,-26.715233,0.0,0.0,0.0,0.140219,0.140219,0.140219,1.0,-1.362340,1.0,-3.436967,29.833747


In [33]:
net.res_trafo

,p_hv_mw,q_hv_mvar,p_lv_mw,q_lv_mvar,pl_mw,ql_mvar,i_hv_ka,i_lv_ka,vm_hv_pu,va_hv_degree,vm_lv_pu,va_lv_degree,loading_percent
0,50.0,0.0,-50.0,0.0,0.0,0.0,0.131216,0.262432,1.0,0.0,1.0,-3.436967,50.0


In [34]:
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,min_p_mw,max_p_mw,power_station_trafo
0,None,2,80.0,1.01,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,80.0,NaN
1,None,3,100.0,1.01,NaN,NaN,NaN,1.0,False,True,0.0,None,True,0.0,100.0,NaN


In [35]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,8.511254,14.859011,-5.708566,1.0
1,81.488747,-9.172781,-1.362340,1.0


In [36]:
net.res_cost

1605.5337611101422

Piecewise linear cost functions

In [37]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,10.0,0.0,0.0,0.0,0.0
1,0,gen,0.0,15.0,0.0,0.0,0.0,0.0
2,1,gen,0.0,12.0,0.0,0.0,0.0,0.0


In [38]:
net.poly_cost.drop(net.poly_cost.index.values, inplace=True)

In [39]:
net.gen[["min_p_mw", "max_p_mw"]]

,min_p_mw,max_p_mw
0,0.0,80.0
1,0.0,100.0


In [40]:
net.ext_grid[["min_p_mw", "max_p_mw"]]

,min_p_mw,max_p_mw
0,-1000.0,1000.0


We define the piecewise linear cost as constant over the whole range to reproduce the polyomial costs defined above:


In [41]:
pp.create_pwl_cost(net, 0, "gen", [[0, 80, 15]])
pp.create_pwl_cost(net, 1, "gen", [[0, 100, 12]])
pp.create_pwl_cost(net, 0, "ext_grid", [[-1000, 1000, 10]])

2

In [42]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,8.511254,14.859011,-5.708566,1.0
1,81.488747,-9.172781,-1.362340,1.0


In [43]:
net.res_cost

1605.5337611101422

In [44]:
pp.rundcopp(net, delta=1e-16)

In [45]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,8.511254,14.859011,-5.708566,1.0
1,81.488747,-9.172781,-1.362340,1.0


In [46]:
net.res_cost

1605.5337611101422

Now lets define real piecewise linear costs for generator 1. We define the costs as 12€/MW up to 70MW and 20€/MW from 70MW to 100MW:

In [47]:
net.pwl_cost.points.loc[1] = [[0, 70, 12], [70, 100, 20]]

In [48]:
pp.rundcopp(net, delta=1e-16)

In [49]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,20.000007,14.859011,-5.220652,1.0
1,69.999997,-9.172781,-1.641146,1.0


In [50]:
pp.runopp(net, verbose=True, delta=1e-16)

PYPOWER Version 5.1.4, 27-June-2018 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 1.60 seconds
Objective Function Value = 1666.07 $/hr
| PyPower (ppci) System Summary - these are not valid for pandapower DataFrames|

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              4     Total Gen Capacity    1180.0       -3000000000.0 to 3000000000.0
Generators         3     On-line Capacity      1180.0       -3000000000.0 to 3000000000.0
Committed Gens     3     Generation (actual)    141.7               1.0
Loads              3     Load                   140.0               0.0
  Fixed            3       Fixed                140.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           4   

In [1]:
from julia.PowerModels import run_ac_opf